In [1]:
import math
import sympy
import mpmath
import random
import primefac
import numpy as np
import scipy.linalg
import qutip
import vpython

def scalar_to_vector(z):
    if z == float('inf'):
        return [0,0,1]
    x = z.real
    y = z.imag
    return [(2*x)/(1.+(x**2)+(y**2)), \
           (2*y)/(1.+(x**2)+(y**2)), \
           (-1.+(x**2)+(y**2))/(1.+(x**2)+(y**2))]

def vector_to_scalar(x, y, z):
    if z == 1:
        return float('inf') 
    else:
        return complex(x/(1-z), y/(1-z))

def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
       return v
    return v / norm
    
def scalar_product(m, n):
    return 0.5*np.trace(np.dot(np.conjugate(m).T, n))

def pauli_decompose(m):
    return normalize(np.array([scalar_product(np.eye(2), m).real,\
                              scalar_product(qutip.sigmax().full(), m).real,\
                              scalar_product(qutip.sigmay().full(), m).real,\
                              scalar_product(qutip.sigmaz().full(), m).real]))
            
    
def mobius_transformation(a, b, c, d):
    def t(z):
        if a*d-b*c == 0:
            return z
        if c != 0:
            if z == -1*d/c:
                return float('inf')
            if z == float('inf'):
                return a/c
        if c == 0:
            if z == float('inf'):
                return float('inf')
        return (a*z + b)/(c*z + d)
    return lambda z: z+t(z)
        
class Sphere:
    def __init__(self, n, dt=0.001):
        self.t = 0
        self.n = n
        self.dt = dt
        self.state = qutip.rand_ket(self.n)
        self.energy = qutip.rand_herm(self.n)
        self.dims = None
        self.scale = None
        self.polynomial = None
        self.pure_scalars = None
        self.pure_vectors = None
        self.pure_matrices = None
        self.pure_energy = None
        self.impure_vectors = None
        self.impure_matrices = None
        self.impure_energies = None
        self.observer = None
        self.old_observer = None
        self.shatter()

    def shatter(self):
        self.polynomial = self.state.full().T.tolist()[0]
        self.scale = self.polynomial[0]
        self.polynomial = [term/self.scale for term in self.polynomial]
        try:
            self.pure_scalars = mpmath.polyroots(self.polynomial)
            self.pure_scalars = [complex(root) for root in self.pure_scalars]
        except:
            print(self.pure_scalars)
        self.pure_vectors = [scalar_to_vector(scalar) for scalar in self.pure_scalars]        
        self.pure_matrices = [vector[0]*qutip.sigmax().full() + \
                              vector[1]*qutip.sigmay().full() + \
                              vector[2]*qutip.sigmaz().full() 
                                for vector in self.pure_vectors]
        self.energy.dims = [[self.n], [self.n]]
        self.state.dims = [[self.n], [1]]
        self.pure_energy = qutip.expect(self.energy, self.state)
        pfacs = list(primefac.primefac(self.n))
        twos = pfacs.count(2)
        remainder = int(self.n/2**twos)
        splitting = [2]*twos
        if remainder != 1:
            self.state.dims = [splitting+[remainder],[1]*(len(splitting)+1)]
        else:
            self.state.dims = [splitting,[1]*(len(splitting))]
        self.dims = self.state.dims[0]
        if self.observer != None:
            self.old_observer = self.impure_matrices[self.observer]
        self.impure_matrices = [self.state.ptrace(i) for i in range(len(splitting))]
        self.energy.dims = [self.dims, self.dims]
        self.impure_energies = [self.energy.ptrace(i) for i in range(len(splitting))]
        self.energy.dims = [[self.n], [self.n]]
        self.impure_energies = [qutip.expect(self.impure_energies[i], self.impure_matrices[i]) for i in range(len(self.impure_matrices))]
        self.impure_vectors = [pauli_decompose(matrix.full()) for matrix in self.impure_matrices]
        
    def mobius_transform(self):
        if self.observer != None:
            if self.old_observer == None:
                self.old_observer = self.impure_matrices[self.observer]
            new_observer = self.impure_matrices[self.observer]
            change = self.old_observer.dag()*new_observer*self.old_observer
            matrix = scipy.linalg.expm(-2*math.pi*complex(0,1)*change.full()*self.dt)
            M = mobius_transformation(matrix[0,0],\
                                      matrix[0,1],\
                                      matrix[1,0],\
                                      matrix[1,1])
            self.pure_scalars = [M(root) for root in self.pure_scalars] 
            x = sympy.symbols("x")
            polynomial = sympy.Poly(reduce(lambda a, b: a*b, [x+root for root in self.pure_scalars]), domain="CC")
            self.polynomial = polynomial.coeffs()
            self.state = qutip.Qobj(np.array(self.polynomial).T)
    
    def hamiltonian_transform(self):
        propagator = scipy.linalg.expm(-2*math.pi*complex(0,1)*self.energy.full()*self.dt)
        self.state = qutip.Qobj(propagator.dot(self.state.full()))
        
    def evolve(self):  
        self.hamiltonian_transform()
        self.mobius_transform()
        self.shatter()
        self.t += self.dt

################
sphere = Sphere(8)
################

vpython.scene.range = 1
vpython.scene.forward = vpython.vector(random.random(), random.random(), random.random())
vpython.scene.lights = []
vsphere = vpython.sphere(pos=vpython.vector(0,0,0), radius=1, color=vpython.color.blue, opacity=0.5)
vpure_stars = [vpython.sphere(pos=vpython.vector(*vector),\
                              radius=0.1, color=vpython.color.white,\
                              opacity=1.0, emissive=True) 
                                     for vector in sphere.pure_vectors]
vimpure_stars = [vpython.sphere(pos=vpython.vector(*vector[1:]),\
                                radius=0.1, color=vpython.color.red,\
                                opacity=1.0, emissive=True) 
                                     for vector in sphere.impure_vectors]

def draw_stars():
    vpython.rate(15)
    for i in range(len(vpure_stars)):
        vpure_stars[i].pos = vpython.vector(*sphere.pure_vectors[i])
        vpure_stars[i].color = vpython.color.hsv_to_rgb(vpython.vector(sphere.pure_energy**2,1,1))
    for i in range(len(vimpure_stars)):
         vimpure_stars[i].pos = vpython.vector(*sphere.impure_vectors[i][1:])
         vimpure_stars[i].color = vpython.color.hsv_to_rgb(vpython.vector(float(sphere.impure_vectors[i][0]),1,1))

def click(event):
    global sphere
    global vimpure_stars
    star = vpython.scene.mouse.pick
    if star in vimpure_stars:
        if star.emissive == True:
            for other in vimpure_stars:
                other.emissive = True
            star.emissive = False
        sphere.observer = vimpure_stars.index(star)

vpython.scene.bind('click', click)
 
while True:
    draw_stars()
    sphere.evolve()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ValueError: Buffer has wrong number of dimensions (expected 1, got 0)